In [1]:
import pandas as pd
scores = pd.read_csv('/home/kieran/raid/Github/Epistasis_MK2/Revisions_Omicron_Epistasis.csv',index_col=0)
scores = scores.reset_index()
scores.significant_masked_change = scores.significant_masked_change.fillna(0)
scores.significant_masked_ratio = scores.significant_masked_ratio.fillna(0)
scores

,source_sequence,target_sequence,source_logits,target_logits,change,ratio,position,masked_change,masked_ratio,significant_masked_change_local,significant_masked_ratio_local,mutation,significant_masked_change,significant_masked_ratio,significant_masked_change_rounded
0,M,M,-0.000460,-0.000454,-0.000006,-0.000006,1,-0.000006,-0.000006,NaN,NaN,A67V,0.0,0.0,NaN
1,F,F,-0.147081,-0.148025,0.000814,0.000944,2,0.000814,0.000944,NaN,NaN,A67V,0.0,0.0,NaN
2,V,V,-0.231966,-0.231425,-0.000428,-0.000540,3,-0.000428,-0.000540,NaN,NaN,A67V,0.0,0.0,NaN
3,F,F,-0.094124,-0.097257,0.002847,0.003133,4,0.002847,0.003133,NaN,NaN,A67V,0.0,0.0,NaN
4,L,L,-0.023677,-0.023800,0.000120,0.000123,5,0.000120,0.000123,NaN,NaN,A67V,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38185,K,K,-0.109476,-0.109899,0.000379,0.000423,1269,0.000379,0.000423,NaN,NaN,L981F,0.0,0.0,NaN
38186,L,L,-0.521582,-0.523057,0.000875,0.001475,1270,0.000875,0.001475,NaN,NaN,L981F,0.0,0.0,NaN
38187,H,H,-0.850460,-0.850260,-0.000086,-0.000200,1271,-0.000086,-0.000200,NaN,NaN,L981F,0.0,0.0,NaN
38188,Y,Y,-0.697745,-0.698370,0.000311,0.000625,1272,0.000311,0.000625,NaN,NaN,L981F,0.0,0.0,NaN


In [2]:
input_file = '/home/kieran/raid/Github/ESM2_Paper_Revision_Data/PDB/6vsb_1_1_1.pdb'

In [3]:
def edit_bfactor(input_file,scores,attribute,is_mutations=False,make_positive=False):
    all_mutation_scores = []
    if is_mutations:
        for mutation in scores.mutation.unique():
            mutation_scores = scores[scores.mutation == mutation]
            all_mutation_scores.append(mutation_scores)
    else:
        all_mutation_scores.append(scores)
    for mutation_scores in all_mutation_scores:
        if is_mutations:
            mutation = mutation_scores.mutation.values[0]
        else:
            mutation = 'reference'

        if 'pos' in mutation_scores.columns:
            position_col = 'pos'
        else:
            position_col = 'position'

        lines = []
        with open(input_file, 'r') as file:
            for l in file:
                if 'ATOM' in l:
                    residue_number = int(l[22:26].split()[0])
                    change = float(mutation_scores[mutation_scores[position_col] == residue_number][attribute].values[0])
                    if make_positive:
                        change = abs(change)
                    l = l[:60] + str(round(change,2))[:6] +l[66:]
                lines.append(l)

            output_file = input_file.split('.')[0]+'_'+mutation+'_'+attribute+'_'+input_file.split('.')[1]+'.pdb'
            with open(output_file, 'w') as file:    
                for l in lines:
                    file.write(l)
                    file.write('\n')

In [4]:
scores.mutation.unique()

array(['A67V', 'T95I', 'G142D', 'L212I', 'G339D', 'S371L', 'S373P',
       'S375F', 'K417N', 'N440K', 'G446S', 'S477N', 'T478K', 'E484A',
       'Q493R', 'G496S', 'Q498R', 'N501Y', 'Y505H', 'T547K', 'D614G',
       'H655Y', 'N679K', 'P681H', 'N764K', 'D796Y', 'N856K', 'Q954H',
       'N969K', 'L981F'], dtype=object)

In [5]:
from Bio.PDB import *

In [23]:
edit_bfactor(input_file,scores,'significant_masked_change',is_mutations=True,make_positive=False)

# Make Annotations

In [6]:
mutations = ['E484A','D614G','N969K']

In [7]:
import ast
import numpy as np

In [16]:
mutation_scores = scores[scores.mutation.isin(['E484A'])]
mutation_scores[mutation_scores.significant_masked_change_local > 0]

,source_sequence,target_sequence,source_logits,target_logits,change,ratio,position,masked_change,masked_ratio,significant_masked_change_local,significant_masked_ratio_local,mutation,significant_masked_change,significant_masked_ratio,significant_masked_change_rounded
16878,P,P,-1.213507,-1.258423,0.013051,0.044915,330,0.013051,0.044915,0.013051,0.044915,E484A,0.013051,0.044915,0.0131
16932,P,P,-0.568238,-0.597801,0.016503,0.029564,384,0.016503,0.029564,0.016503,0.029564,E484A,0.016503,0.029564,0.0165
17011,P,P,-0.489235,-0.503221,0.008515,0.013986,463,0.008515,0.013986,0.008515,NaN,E484A,0.000000,0.000000,NaN
17013,E,E,-0.585588,-0.597927,0.006828,0.012339,465,0.006828,0.012339,0.006828,NaN,E484A,0.000000,0.000000,NaN
17015,D,D,-0.415108,-0.426512,0.007487,0.011403,467,0.007487,0.011403,0.007487,NaN,E484A,0.000000,0.000000,NaN
17024,G,G,-0.703185,-0.743707,0.019658,0.040522,476,0.019658,0.040522,0.019658,0.040522,E484A,0.019658,0.040522,0.0197
17025,N,N,-0.506923,-0.519588,0.007581,0.012665,477,0.007581,0.012665,0.007581,NaN,E484A,0.000000,0.000000,NaN
17029,N,N,-0.524482,-0.558124,0.019580,0.033642,481,0.019580,0.033642,0.019580,0.033642,E484A,0.019580,0.033642,0.0196
17030,G,G,-0.391240,-0.444974,0.035377,0.053734,482,0.035377,0.053734,0.035377,0.053734,E484A,0.035377,0.053734,0.0354
17033,G,G,-0.332300,-0.381326,0.034317,0.049027,485,0.034317,0.049027,0.034317,0.049027,E484A,0.034317,0.049027,0.0343


In [8]:
mutation_scores = scores[scores.mutation.isin(mutations)]

In [53]:
command = 'hide atoms;hide /B cartoon;hide /C cartoon;'
for i,mutation in enumerate(mutations):
    print(mutation)
    mutation_list = mutation.split(',')
    hashtag = '#'+str(i+1)
    sites = mutation_scores[(abs(mutation_scores['significant_masked_change']) > 0) & (mutation_scores.mutation == mutation) ].position.values.astype(str)
    command +='select '+hashtag+'/A:'+','.join(sites)+';show sel surfaces;show sel atoms;'
command +='select subtract sel;set bgColor white;lighting flat;lighting shadows true intensity 0.5;color by bfactor palette -0.35,darkred:-0.05,red:-0.01,salmon:0,silver:0.01,cornflowerblue:0.05,blue:0.35,darkblue;tile;transparency 35 surface;'

for i,mutation in enumerate(mutations):
    mutation_list = mutation.split(',')
    hashtag = '#'+str(i+1)
    sites = [s[1:-1] for s in mutation_list]
    command +='select '+hashtag+'/A:'+','.join(sites)+';show sel surfaces;color sel yellow;lighting flat;'
# pyperclip.copy(command)

E484A
D614G
N969K


In [55]:
command

'hide atoms;hide /B cartoon;hide /C cartoon;select #1/A:330,384,444,458,462,466,476,478,481,482,485,486,488,491,493,498,500,509,510,513,521,544;show sel surfaces;show sel atoms;select #2/A:330,488,500,513,515,521,525,526,537,561,564,575,577,586,592,593,597,600,601,604,607,608,609,611,612,613,615,616,617,619,620,622,623,631,634,642,646,649,660,692,697,699,723,724,728,739;show sel surfaces;show sel atoms;select #3/A:330,708,734,739,764,774,792,836,855,904,909,917,933,943,948,961,963,964,965,967,968,976,978,982,1003,1005,1007,1013,1021,1052,1058,1065,1072,1074,1080,1083,1092,1119,1143,1167,1180,1181,1202,1223;show sel surfaces;show sel atoms;select subtract sel;set bgColor white;lighting flat;lighting shadows true intensity 0.5;color by bfactor palette color by bfactor palette -0.35,darkred:-0.05,red:-0.01,salmon:0,silver:0.01,cornflowerblue:0.05,blue:0.35,darkblue;tile;transparency 35 surface;select #1/A:484;show sel surfaces;color sel yellow;lighting flat;select #2/A:614;show sel surfac